# RW API and visualising data in Carto

Exploring code in the RW platform, you can see that there are id codes for layers in the `div` itesm on the sites.

Layers are visulisations on a dataset.

For example, the VIIRS fires dataset is here: 

https://api.resourcewatch.org/v1/dataset/20cc5eca-8c63-4c41-8e8e-134dcf1e6d76

And a layer view on that dataset is here (based on the id exposed in the RW site).

http://api.resourcewatch.org/v1/layer/f9029b9a-a435-4d5f-9800-7440db013ada

You can see that the Layer refrences the dataset id. It contains all the info to produce tiles.

You can also search for all layers that use a particular dataset id, e.g.:
https://api.resourcewatch.org/v1/layer?dataset=20cc5eca-8c63-4c41-8e8e-134dcf1e6d76

To display a layer on a map, you will need to use attributes from the layer config, and also call the Carto map service, and then handle the response to create a tile url object.

```javascript
const layerTpl = {
  version: '1.3.0',
  stat_tag: 'API',
  layers: layer.body.layers.map((l) => {
    if (isInteractive) {
      return {
        ...l,
        options: {
          ...l.options,
          interactivity: layerSpec.interactionConfig.output.map(o => o.column)
        }
      };
    }
    return l;
  })
};
const params = `?stat_tag=API&config=${encodeURIComponent(JSON.stringify(layerTpl))}`;

fetch(`https://${layer.account}.carto.com/api/v1/map${params}`)
  .then((response) => {
    this.errors = !response.ok;
    if (this.errors) this.rejectLayersLoading = true;
    return response.json();
  })
  .then((data) => {
    if (verifyLayersOnly === true) {
      if (this.layersUpdated && typeof this.layersUpdated === 'function') this.layersUpdated(!this.errors, data);
      if (this.errors) this.errorDetails = data;
      if (callback && typeof callback === 'function') callback(!this.errors);
      return;
    }

    const tileUrl = `${data.cdn_url.templates.https.url}/${layer.account}/api/v1/map/${data.layergroupid}/{z}/{x}/{y}.png`;
```

In [1]:
import folium
import json
#import urllib
import requests
from pprint import pprint

### Example of how to generate web map tiles from a carto layer using the RW api

In [ ]:
account = 'wri-rw'
urlCarto = 'https://'+account+'.carto.com/api/v1/map'
body = {
    "layers": [{
        "type": "cartodb",
        "options": {
            "sql": "select * from countries",
            "cartocss":"#layer {\n  polygon-fill: #374C70;\n  polygon-opacity: 0.9;\n  polygon-gamma: 0.5;\n  line-color: #FFF;\n  line-width: 1;\n  line-opacity: 0.5;\n  line-comp-op: soft-light;\n}",
            "cartocss_version": "2.1.1"
        }
    }]
}

In [ ]:
r  =  requests.post(urlCarto, data=json.dumps(body), headers={'content-type': 'application/json; charset=UTF-8'})
tileUrl = 'https://'+account+'.carto.com/api/v1/map/' + r.json()['layergroupid'] + '/{z}/{x}/{y}.png32';

In [ ]:
r.json()

In [ ]:
map_osm = folium.Map(location=[45.5236, 0.6750], zoom_start=3)
folium.TileLayer(
    tiles=tileUrl,
    attr='text',
    name='text',
    overlay=True
).add_to(map_osm)
map_osm

### Replicate this for a random layer

In [2]:
layer_url = 'http://api.resourcewatch.org/v1/layer/f9029b9a-a435-4d5f-9800-7440db013ada'
r = requests.get(layer_url)
data = r.json().get('data')
pprint(data)

{'attributes': {'application': ['rw'],
                'applicationConfig': {},
                'dataset': '20cc5eca-8c63-4c41-8e8e-134dcf1e6d76',
                'default': False,
                'description': 'This layer shows fires detected in the past '
                               'week.',
                'env': 'production',
                'interactionConfig': {'output': [{'column': 'frp',
                                                  'format': None,
                                                  'prefix': '',
                                                  'property': 'Fire Radiative '
                                                              'Power',
                                                  'suffix': '',
                                                  'type': 'number'},
                                                 {'column': 'acq_date',
                                                  'format': None,
                                             

In [38]:
data.get('attributes').get('layerConfig').get('account')

'wri-01'

In [ ]:
{
  "version": "1.3.0",
  "layers": [{
    "type": "mapnik",
    "options": {
      "cartocss_version": "2.1.1",
      "cartocss": "#layer { polygon-fill: #FFF; }",
      "sql": "select * from european_countries_e",
      "interactivity": ["cartodb_id", "iso3"]
    }
  }]
}

In [37]:
data

{'id': 'f9029b9a-a435-4d5f-9800-7440db013ada',
 'type': 'layer',
 'attributes': {'name': 'All Fires (Past Week)',
  'slug': 'All-Fires-Latest-Week',
  'dataset': '20cc5eca-8c63-4c41-8e8e-134dcf1e6d76',
  'description': 'This layer shows fires detected in the past week.',
  'application': ['rw'],
  'iso': [],
  'provider': 'cartodb',
  'userId': '57d021e329309063404573a8',
  'default': False,
  'protected': False,
  'env': 'production',
  'layerConfig': {'account': 'wri-01',
   'body': {'maxzoom': 18,
    'minzoom': 3,
    'layers': [{'type': 'cartodb',
      'options': {'sql': "SELECT * FROM vnp14imgtdl_nrt_global_7d where (acq_date> (CURRENT_DATE - interval '7 day')) ORDER BY frp asc",
       'cartocss': '#layer { marker-width: 4; marker-fill: #ffcc00; marker-fill-opacity: 1; marker-line-color: #FFF; marker-line-width: 0; marker-line-opacity: 1; marker-placement: point; marker-type: ellipse; marker-allow-overlap: true; }',
       'cartocss_version': '2.3.0'}}],
    'version': '1.3.0'}

In [43]:
# Create a payload to send to Carto using a  response from a Layer Config

layer_account = data.get('attributes').get('layerConfig').get('account')
print(f'Working with Carto account: {layer_account}\n')

body = data.get('attributes').get('layerConfig').get('body')
body["version"]="1.3.0"


pprint(body)

#carto_r = requests.post(f"https://{layer_account}.carto.com/api/v1/map", data=json.dumps(body), headers={'content-type': 'application/json; charset=UTF-8'})

carto_r = requests.post(f"https://{layer_account}.carto.com/api/v1/map", json=body)

print(carto_r.url)
print(carto_r.status_code)
print(carto_r.text)

Working with Carto account: wri-01

{'layers': [{'options': {'cartocss': '#layer { marker-width: 4; marker-fill: '
                                     '#ffcc00; marker-fill-opacity: 1; '
                                     'marker-line-color: #FFF; '
                                     'marker-line-width: 0; '
                                     'marker-line-opacity: 1; '
                                     'marker-placement: point; marker-type: '
                                     'ellipse; marker-allow-overlap: true; }',
                         'cartocss_version': '2.3.0',
                         'sql': 'SELECT * FROM vnp14imgtdl_nrt_global_7d where '
                                "(acq_date> (CURRENT_DATE - interval '7 day')) "
                                'ORDER BY frp asc'},
             'type': 'cartodb'}],
 'maxzoom': 18,
 'minzoom': 3,
 'version': '1.3.0'}
https://wri-01.carto.com/api/v1/map
200
{"layergroupid":"c8b4f7d12c6da28e9466f28acdc7fbc7:1525862343468","met

In [46]:
carto_r.json()

{'layergroupid': 'c8b4f7d12c6da28e9466f28acdc7fbc7:1525862343468',
 'metadata': {'layers': [{'type': 'mapnik',
    'id': 'layer0',
    'meta': {'cartocss': '#layer { marker-width: 4; marker-fill: #ffcc00; marker-fill-opacity: 1; marker-line-color: #FFF; marker-line-width: 0; marker-line-opacity: 1; marker-placement: point; marker-type: ellipse; marker-allow-overlap: true; }',
     'stats': {'estimatedFeatureCount': 231144},
     'cartocss_meta': {'rules': []}},
    'tilejson': {'vector': {'tilejson': '2.2.0',
      'tiles': ['https://cartocdn-ashbu-a.global.ssl.fastly.net/wri-01/api/v1/map/c8b4f7d12c6da28e9466f28acdc7fbc7:1525862343468/layer0/{z}/{x}/{y}.mvt',
       'https://cartocdn-ashbu-b.global.ssl.fastly.net/wri-01/api/v1/map/c8b4f7d12c6da28e9466f28acdc7fbc7:1525862343468/layer0/{z}/{x}/{y}.mvt',
       'https://cartocdn-ashbu-c.global.ssl.fastly.net/wri-01/api/v1/map/c8b4f7d12c6da28e9466f28acdc7fbc7:1525862343468/layer0/{z}/{x}/{y}.mvt',
       'https://cartocdn-ashbu-d.global.s

In [58]:
# #r  =  requests.post(urlCarto, data=json.dumps(body_test), headers={'content-type': 'application/json; charset=UTF-8'})
# tileUrl = (f"https://{layer_account}.carto.com/api/v1/map/{carto_r.json()['layergroupid']}/"
#           "{z}/{x}/{y}.png")

# tileUrl

In [59]:
# if we were able to get a layergroupid back from carto, we could then form a tile request
# in the following way.

cdn_url = 'https://cartocdn-ashbu-a.global.ssl.fastly.net'
layer_account = data.get('attributes').get('layerConfig').get('account')

tiles = (f"{cdn_url}/{layer_account}/api/v1/map/{carto_r.json()['layergroupid']}"
          "/{z}/{x}/{y}.png")
print(tiles)

https://cartocdn-ashbu-a.global.ssl.fastly.net/wri-01/api/v1/map/c8b4f7d12c6da28e9466f28acdc7fbc7:1525862343468/{z}/{x}/{y}.png


In [66]:
m2 = folium.Map(location=[-10.141931686131018, -55.283203125],zoom_start=4, tiles='Open Street Map')
m2.add_tile_layer(tiles=tiles, max_zoom=12, min_zoom=1,  attr='Custom tiles')

m2

## Next part - interactivity

This is done via the UTFJson
https://github.com/danzel/Leaflet.utfgrid and is already present
